# 1.1 Test original yolov5s model with test_train.mp4 

In [5]:
!python detect.py --source Urban_perception/train/train_video.mp4 --weights yolov5s.pt --classes 0 1 2 3 5 7 9 11

2025-12-09 13:35:32.548119707 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
detect: weights=['yolov5s.pt'], source=Urban_perception/train/train_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=[0, 1, 2, 3, 5, 7, 9, 11], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-

# 1.2 measure accuracy

In [4]:
!python val.py --weights yolov5s.pt --batch-size 1

2025-12-12 13:31:22.388512438 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
val: data=data/coco128.yaml, weights=['yolov5s.pt'], batch_size=1, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING ⚠️ requirements: ❌ C

# 1.3 Measure speed
 - Measure the direct inference time and the total time, including pre- and post-processing

In [4]:
import torch
import cv2
import time
import numpy as np
from pathlib import Path
from utils.augmentations import letterbox
from utils.general import non_max_suppression, scale_boxes
from models.common import DetectMultiBackend
import warnings
warnings.filterwarnings('ignore') 

video_path = 'Urban_perception/train/train_video.mp4'
weights = 'yolov5s.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current Device : {device}")

model = DetectMultiBackend(weights, device=device, dnn=False)
#### "pt" check if the model format is pytorch.
stride, names, pt = model.stride, model.names, model.pt
imgsz = (640,640)
print("Model Loaded")

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Video Path Error {video_path}") 
else:
    print("Warning GPU...")
    # dummy_input = torch.zeros((1, 3, 640, 640), device='cuda')
    for _ in range(3):
        model.warmup(imgsz=(1,3, *imgsz))
        # model(dummy_input)
    print("Warm-up Done")

    time_logs = {'total':[], 'inference':[]}

    print("Start Profiling...")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        torch.cuda.synchronize()
        t_start = time.time()
        im = letterbox(frame, imgsz, stride=stride, auto=pt)[0]

        im = im.transpose((2, 0, 1))[::-1]
        im = np.ascontiguousarray(im)
        im = torch.from_numpy(im).to(device)
        im = im.float()
        im /= 255.0

        # 배치 차원 추가 (1, 3, 640, 640)
        if len(im.shape) == 3:
            im = im[None]

        # 2. 순수 추론 시간 측정 (이제 '이미지'가 아니라 '텐서'가 들어감)
        infer_start = time.time()
        pred = model(im)  # 여기서 순수 추론만 수행됨!
        torch.cuda.synchronize()
        infer_end = time.time()

        pred = non_max_suppression(pred, 0.5, 0.45)

        torch.cuda.synchronize()
        t_end = time.time()

        frame_time = (t_end - t_start) * 1000
        time_logs['total'].append(frame_time)
        infer_time = (infer_end - infer_start) * 1000
        time_logs['inference'].append(infer_time)
        

        if cv2.waitKey(1) == ord('q'):
            break    

    cap.release()
    cv2.destroyAllWindows()

    if len(time_logs['total']) > 0:
        avg_time = sum(time_logs['total']) / len(time_logs['total'])
        infer_times = sum(time_logs['inference'])/len(time_logs['inference'])
        fps = 1000/avg_time
        print(f"📊 Profiling Result Summary")
        print("="*40)
        print(f"Average Process Time : {avg_time:.2f} ms")
        print(f"Inference Time : {infer_times:.2f} ms")
        print(f"Estimated FPS        : {fps:.2f} FPS")
        print("="*40)
    else:
        print("No frames processed.")

Current Device : cuda


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


Model Loaded
Warning GPU...
Warm-up Done
Start Profiling...
📊 Profiling Result Summary
Average Process Time : 34.55 ms
Inference Time : 24.51 ms
Estimated FPS        : 28.95 FPS


# 1.4 Profiling using nsys with .pt model

In [ ]:
# profiling for 60s to collect power and tmeperature with tegrastats
!/opt/nvidia/nsight-systems/2024.5.4/target-linux-tegra-armv8/nsys profile \
-t cuda,osrt,nvtx \
--delay=20 \
--duration=60 \
-o ori_yolov5s_60s \
--force-overwrite true \
python detect.py --weights yolov5s.pt --source Urban_perception/train/train_video.mp4 --classes 0 1 2 3 5 7 9 11

2025-12-12 23:45:35.115242478 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
detect: weights=['yolov5s.pt'], source=Urban_perception/train/train_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=[0, 1, 2, 3, 5, 7, 9, 11], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-

# 2.Convert yolov5s.pt to ONNX

In [ ]:
# --include : save as onnx 
# --simplyfy : simplify unnecessary computations
!python export.py --weights yolov5s.pt --include onnx --simplify

2025-12-10 01:13:03.712820233 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
export: data=data/coco128.yaml, weights=['yolov5s.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=True, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 2025-10-19 Python-3.10.12 torch-2.4.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs

PyTorch: starting from yolov5s.pt with output shape (1, 25200, 85) (14.1 MB)
requirements: Ultralytics requirement ['onnxscript'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir

# 2.1 Convert .onnx file to TRT engine with 32bit

In [1]:
!trtexec --onnx=yolov5s.onnx --saveEngine=yolov5s.engine

&&&& RUNNING TensorRT.trtexec [TensorRT v100400] [b26] # trtexec --onnx=yolov5s.onnx --saveEngine=yolov5s.engine
[12/17/2025-03:20:20] [I] === Model Options ===
[12/17/2025-03:20:20] [I] Format: ONNX
[12/17/2025-03:20:20] [I] Model: yolov5s.onnx
[12/17/2025-03:20:20] [I] Output:
[12/17/2025-03:20:20] [I] === Build Options ===
[12/17/2025-03:20:20] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default, tacticSharedMem: default
[12/17/2025-03:20:20] [I] avgTiming: 8
[12/17/2025-03:20:20] [I] Precision: FP32
[12/17/2025-03:20:20] [I] LayerPrecisions: 
[12/17/2025-03:20:20] [I] Layer Device Types: 
[12/17/2025-03:20:20] [I] Calibration: 
[12/17/2025-03:20:20] [I] Refit: Disabled
[12/17/2025-03:20:20] [I] Strip weights: Disabled
[12/17/2025-03:20:20] [I] Version Compatible: Disabled
[12/17/2025-03:20:20] [I] ONNX Plugin InstanceNorm: Disabled
[12/17/2025-03:20:20] [I] TensorRT runtime: full
[12/17/2025-03:20:20] [I] Lean DLL Path: 
[12/17/2025

[12/17/2025-03:20:20] [I] Selected Device: Orin
[12/17/2025-03:20:20] [I] Selected Device ID: 0
[12/17/2025-03:20:20] [I] Selected Device UUID: GPU-d24e5545-2723-5235-b437-7a4635ceff89
[12/17/2025-03:20:20] [I] Compute Capability: 8.7
[12/17/2025-03:20:20] [I] SMs: 8
[12/17/2025-03:20:20] [I] Device Global Memory: 7619 MiB
[12/17/2025-03:20:20] [I] Shared Memory per SM: 164 KiB
[12/17/2025-03:20:20] [I] Memory Bus Width: 128 bits (ECC disabled)
[12/17/2025-03:20:20] [I] Application Compute Clock Rate: 1.02 GHz
[12/17/2025-03:20:20] [I] Application Memory Clock Rate: 0.612 GHz
[12/17/2025-03:20:20] [I] 
[12/17/2025-03:20:20] [I] Note: The application clock rates do not reflect the actual clock rates that the GPU is currently running at.
[12/17/2025-03:20:20] [I] 
[12/17/2025-03:20:20] [I] TensorRT version: 10.4.0
[12/17/2025-03:20:20] [I] Loading standard plugins
[12/17/2025-03:20:21] [I] [TRT] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 30, GPU 6237 (MiB)
[12/17/2025-03:20:24]

# 2.2 Measure accuracy

In [7]:
!python val.py --weights yolov5s.engine

2025-12-16 05:23:12.366930209 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
val: data=data/coco128.yaml, weights=['yolov5s.engine'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING ⚠️ requirements

# 2.3 Measure speed
 - Measure the direct inference time and the total time, including pre- and post-processing

In [5]:
import torch
import cv2
import time
import numpy as np
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore') 
from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes
from utils.augmentations import letterbox

yolov5_path = '/workspace/yolov5-master'
sys.path.append(yolov5_path)

video_path = 'Urban_perception/train/train_video.mp4'
weights = 'yolov5s.engine'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current Device : {device}")

#### DetectMultiBackend can load wahtever the model shape is, such as .pt, .engine and onnx.
#### A parameter dnn means using TRT and Pytorch engine.
# load engine model 
model = DetectMultiBackend(weights, device=device, dnn=False)
#### "pt" check if the model format is pytorch.
stride, names, pt = model.stride, model.names, model.pt
imgsz = (640,640)
print("Model Loaded")

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Video Path Error {video_path}") 
else:
    print("Warning Engine..")
    # warmup the GPU    
    for _ in range(3):
        model.warmup(imgsz=(1,3, *imgsz))
    print("Warm-up Done")
    time_logs = {'total':[], 'inference':[]}
    print("Start profiling engine...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        torch.cuda.synchronize()
        t_start = time.time()

        #pre-processing
        im = letterbox(frame, imgsz, stride=stride, auto=pt)[0]
        # resizing (H,W,C) -> (C,H,W)
        im = im.transpose((2,0,1))[::-1]
        # reorder the data according to the new sequence to improve performance
        im = np.ascontiguousarray(im)
        im = torch.from_numpy(im).to(device)
        im = im.float()
        im /= 255.0
        # add batch dimension 
        if len(im.shape) == 3:
            im = im[None]

        #inference
        infer_start = time.time()
        pred = model(im)
        torch.cuda.synchronize()
        infer_end = time.time()

        #post-processing
        # remove overlapping boxes 
        pred = non_max_suppression(pred, 0.5, 0.45)

        torch.cuda.synchronize()
        t_end = time.time()

        frame_time = (t_end - t_start) * 1000
        time_logs['total'].append(frame_time)
        infer_time = (infer_end - infer_start) * 1000
        time_logs['inference'].append(infer_time)
        

        if cv2.waitKey(1) == ord('q'):
            break    

    cap.release()
    cv2.destroyAllWindows()

    if len(time_logs['total']) > 0:
        avg_time = sum(time_logs['total']) / len(time_logs['total'])
        infer_times = sum(time_logs['inference'])/len(time_logs['inference'])
        fps = 1000/avg_time
        print(f"📊 Profiling Result Summary")
        print("="*40)
        print(f"Average Process Time : {avg_time:.2f} ms")
        print(f"Inference Time : {infer_times:.2f} ms")
        print(f"Estimated FPS        : {fps:.2f} FPS")
        print("="*40)
    else:
        print("No frames processed.")

Loading yolov5s.engine for TensorRT inference...


Current Device : cuda
[12/17/2025-12:38:54] [TRT] [I] Loaded engine size: 29 MiB
[12/17/2025-12:38:54] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[12/17/2025-12:38:54] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +33, now: CPU 0, GPU 60 (MiB)
Model Loaded
Warning Engine..
Warm-up Done
Start profiling engine...
📊 Profiling Result Summary
Average Process Time : 39.26 ms
Inference Time : 24.66 ms
Estimated FPS        : 25.47 FPS


# 2.4 Profiling using nsys and detect.py 

In [13]:
# profiling for 60s to collect power and tmeperature with tegrastats
!/opt/nvidia/nsight-systems/2024.5.4/target-linux-tegra-armv8/nsys profile \
-t cuda,osrt,nvtx \
--delay=20 \
--duration=60 \
-o ori_yolov5s_engine_60s \
--force-overwrite true \
python detect.py --weights yolov5s.engine --source Urban_perception/train/train_video.mp4 --classes 0 1 2 3 5 7 9 11

2025-12-16 05:50:47.284539476 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
detect: weights=['yolov5s.engine'], source=Urban_perception/train/train_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=[0, 1, 2, 3, 5, 7, 9, 11], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned 

# 3.1  Convert .onnx file to TRT engine with 16bit

In [ ]:
!trtexec --onnx=yolov5s.onnx --fp16 --saveEngine=yolov5s_FP16.engine

&&&& RUNNING TensorRT.trtexec [TensorRT v100400] [b26] # trtexec --onnx=yolov5s.onnx --fp16 --saveEngine=yolov5s_FP16.engine
[12/17/2025-03:23:53] [I] === Model Options ===
[12/17/2025-03:23:53] [I] Format: ONNX
[12/17/2025-03:23:53] [I] Model: yolov5s.onnx
[12/17/2025-03:23:53] [I] Output:
[12/17/2025-03:23:53] [I] === Build Options ===
[12/17/2025-03:23:53] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default, tacticSharedMem: default
[12/17/2025-03:23:53] [I] avgTiming: 8
[12/17/2025-03:23:53] [I] Precision: FP32+FP16
[12/17/2025-03:23:53] [I] LayerPrecisions: 
[12/17/2025-03:23:53] [I] Layer Device Types: 
[12/17/2025-03:23:53] [I] Calibration: 
[12/17/2025-03:23:53] [I] Refit: Disabled
[12/17/2025-03:23:53] [I] Strip weights: Disabled
[12/17/2025-03:23:53] [I] Version Compatible: Disabled
[12/17/2025-03:23:53] [I] ONNX Plugin InstanceNorm: Disabled
[12/17/2025-03:23:53] [I] TensorRT runtime: full
[12/17/2025-03:23:53] [I] Lean DLL P

# 3.2 Measure accuracy

In [ ]:
!python val.py --weights yolov5s_FP16.engine 

2025-12-16 05:58:41.995609931 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
val: data=data/coco128.yaml, weights=['yolov5s_FP16.engine'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' returned non-zero exit status 2.
WARNING ⚠️ require

# 3.3 Measure speed
 - Measure the direct inference time and the total time, including pre- and post-processing

In [ ]:
import torch
import cv2
import time
import numpy as np
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore') 
from models.common import DetectMultiBackend
from utils.general import non_max_suppression
from utils.augmentations import letterbox

yolov5_path = '/workspace/yolov5-master'
sys.path.append(yolov5_path)

video_path = 'Urban_perception/train/train_video.mp4'
weights = 'yolov5s_FP16.engine'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current Device : {device}")

#### DetectMultiBackend can load wahtever the model shape is, such as .pt, .engine and onnx.
#### A parameter dnn means using TRT and Pytorch engine.
# load engine model 
model = DetectMultiBackend(weights, device=device, dnn=False)
#### "pt" check if the model format is pytorch.
stride, names, pt = model.stride, model.names, model.pt
imgsz = (640,640)
print("Model Loaded")

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Video Path Error {video_path}") 
else:
    print("Warning Engine..")
    #### ah the first time, GPU is slow since assining memory and loading kernel, so warmup GPU with dummy data
    # warmup the GPU
    for _ in range(3):
        model.warmup(imgsz=(1,3, *imgsz))
    print("Warm-up Done")
    time_logs = {'total':[], 'inference':[]}
    print("Start profiling engine...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        #### The CPU is halted (temporarily stopped) until the GPU finishes its tasks, 
        #### thereby preventing the measurement from starting before the GPU initiates its workload.
        torch.cuda.synchronize()
        t_start = time.time()

        #pre-processing
        #### letterbox performs image resizing by keeping the aspect of ratio and adding padding to the empty spaces.
        im = letterbox(frame, imgsz, stride=stride, auto=pt)[0]
        #### OpenCV reads images in the order of Height, Width and Color(Channel),
        #### but model reads imges in the order of Color, Height and Width.
        #### transpose changes the image shape, (H,W,C) -> (C,H,W).
        # resizing (H,W,C) -> (C,H,W)
        im = im.transpose((2,0,1))[::-1]
        # reorder the data according to the new sequence to improve performance
        im = np.ascontiguousarray(im)
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()
        im /= 255.0
        # add batch dimension 
        if len(im.shape) == 3:
            im = im[None]

        #inference
        infer_start = time.time()
        pred = model(im)
        infer_end = time.time()

        #post-processing
        #### non_max_suppression(NMS) remove boxes that engine drows except for just one box, which has highest probability.
        # remove overlapping boxes 
        pred = non_max_suppression(pred, 0.5, 0.45)

        torch.cuda.synchronize()
        t_end = time.time()

        frame_time = (t_end - t_start) * 1000
        time_logs['total'].append(frame_time)
        infer_time = (infer_end - infer_start) * 1000
        time_logs['inference'].append(infer_time)
        

        if cv2.waitKey(1) == ord('q'):
            break    

    cap.release()
    cv2.destroyAllWindows()

    if len(time_logs['total']) > 0:
        avg_time = sum(time_logs['total']) / len(time_logs['total'])
        infer_times = sum(time_logs['inference'])/len(time_logs['inference'])
        fps = 1000/avg_time
        print(f"📊 Profiling Result Summary")
        print("="*40)
        print(f"Average Process Time : {avg_time:.2f} ms")
        print(f"Inference Time : {infer_times:.2f} ms")
        print(f"Estimated FPS        : {fps:.2f} FPS")
        print("="*40)
    else:
        print("No frames processed.")

Current Device : cuda


Loading yolov5s_FP16.engine for TensorRT inference...


[12/16/2025-05:42:59] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[12/16/2025-05:42:59] [TRT] [I] Loaded engine size: 16 MiB
[12/16/2025-05:42:59] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +16, now: CPU 2, GPU 211 (MiB)
Model Loaded
Warning Engine..
Warm-up Done
Start profiling engine...


📊 Profiling Result Summary
Average Process Time : 30.31 ms
Inference Time : 16.39 ms
Estimated FPS        : 32.99 FPS


# 3.4 Profiling with FP16 enigine using nsys and detect.py

In [11]:
# profiling for 60s to collect power and tmeperature with tegrastats
!/opt/nvidia/nsight-systems/2024.5.4/target-linux-tegra-armv8/nsys profile \
-t cuda,osrt,nvtx \
--delay=20 \
--duration=60 \
-o yolov5s_FP16_engine_60s \
--force-overwrite true \
python detect.py --weights yolov5s_FP16.engine --source Urban_perception/train/train_video.mp4 --classes 0 1 2 3 5 7 9 11

2025-12-16 05:41:33.518652605 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"
detect: weights=['yolov5s_FP16.engine'], source=Urban_perception/train/train_video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=[0, 1, 2, 3, 5, 7, 9, 11], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8"" ' retu

video 1/1 (285/7792) /workspace/yolov5-master/Urban_perception/train/train_video.mp4: 640x640 3 persons, 1 bicycle, 2 cars, 2 buss, 1 truck, 16.1ms
video 1/1 (286/7792) /workspace/yolov5-master/Urban_perception/train/train_video.mp4: 640x640 3 persons, 2 cars, 2 buss, 1 truck, 16.1ms
video 1/1 (287/7792) /workspace/yolov5-master/Urban_perception/train/train_video.mp4: 640x640 4 persons, 4 cars, 1 motorcycle, 1 bus, 2 trucks, 16.4ms
video 1/1 (288/7792) /workspace/yolov5-master/Urban_perception/train/train_video.mp4: 640x640 6 persons, 1 bicycle, 4 cars, 1 motorcycle, 2 buss, 2 trucks, 16.2ms
video 1/1 (289/7792) /workspace/yolov5-master/Urban_perception/train/train_video.mp4: 640x640 2 persons, 1 bicycle, 3 cars, 1 motorcycle, 2 buss, 2 trucks, 16.1ms
video 1/1 (290/7792) /workspace/yolov5-master/Urban_perception/train/train_video.mp4: 640x640 4 persons, 3 cars, 1 bus, 2 trucks, 16.2ms
video 1/1 (291/7792) /workspace/yolov5-master/Urban_perception/train/train_video.mp4: 640x640 5 perso